# forex

<img src = "images/prima.jpg">

#### Il forex (anche noto come 'FX', acronimo di 'Foreign Exchange') è una forma di cambio valutario che consiste nella conversione di una valuta in un'altra. Il forex è uno dei mercati più dinamici al mondo, con transazioni giornaliere da parte di privati, società e banche.

Buona parte dei cambi valutari venga fatta per scopi pratici, ma la maggioranza delle conversioni di valuta viene effettuata da trader forex per ottenere un profitto. L'importo della valuta convertita ogni giorno può rendere l'andamento del prezzo di alcune valute estremamente volatile.

# Coppia Forex
Una coppia forex è una combinazione di due valute che vengono scambiate tra loro. Ci sono centinaia di combinazioni diverse tra cui scegliere. Tra le più popolari ci sono euro contro dollaro statunitense (EUR/USD), dollaro statunitense contro yen giapponese (USD/JPY) e sterlina britannica contro dollaro statunitense (GBP/USD).

![monete](images/monete.png)

La valuta base è sempre indicata a sinistra della coppia valutaria, mentre la valuta quotata è quella sulla destra. La valuta base corrisponde sempre a uno, mentre la valuta quotata è uguale al prezzo quotato attuale della coppia e indica la quantità della valuta quotata necessaria per acquistare un'unità della valuta base. Quindi, facendo trading sulle valute, venderai una valuta per acquistarne un'altra.

![cambio](images/eurusd.png)

# Acquistare o vendere una coppia valutaria

Acquistare una coppia valutaria significa che prevedi un aumento del prezzo di una valuta. Ciò indica un apprezzamento della valuta base rispetto alla valuta quotata. Vendere una coppia valutaria significa che prevedi un ribasso del prezzo, ossia un deprezzamento della valuta base rispetto alla valuta quotata.

Ad esempio, acquisterai (‘buy’) la coppia EUR/USD se prevedi un rafforzamento dell'euro rispetto al dollaro, ovvero saranno necessari più dollari per acquistare un singolo euro. Oppure, venderai (‘sell’) se prevedi un indebolimento dell'euro rispetto al dollaro, ovvero saranno necessari meno dollari per acquistare un singolo euro.

<img src = "images/demo.jpg">

# Obiettivo progetto

L'obiettivo del progetto è di monitorare e stimare la coppia EUR/USD durante le ore del giorno dato che i trader che operano sulle coppie forex traggono profitto dall'apprezzamento o dal deprezzamento di una valuta rispetto a un'altra.


# Tecnologie

<img src = "images/Forex.drawio.png">

# Logstash
 
**Logstash** permette di raccogliere dati provenienti da diverse sorgenti, modificare e inoltrare dati verso una o più destinazioni.  
**http_poller** è un plug-in di input Logstash che consente di chiamare un'API HTTP, decodificarne l'output in eventi e inviarli.  
I dati sono forniti dal sito: https://www.alphavantage.co, le richieste HTTP vengono mandate da Logstash usando il plug-in http_poller.  
I dati della chaimata API vengono inviati al topic "eurusd" di Kafka.  
La risposta API contiene i seguenti dati: 
```

{
    "Realtime Currency Exchange Rate": {
        "1. From_Currency Code": "EUR",
        "2. From_Currency Name": "Euro",
        "3. To_Currency Code": "USD",
        "4. To_Currency Name": "United States Dollar",
        "5. Exchange Rate": "1.07158160",
        "6. Last Refreshed": "2023-06-05 19:06:42",
        "7. Time Zone": "UTC",
        "8. Bid Price": "1.07158111",
        "9. Ask Price": "1.07162982"
    }
}

```
<img src = "images/logstash.png">

# Apache Kafka

Apache Kafka è una piattaforma di streaming di eventi distribuiti open source utilizzata da migliaia di aziende per pipeline di dati ad alte prestazioni, analisi di streaming, integrazione dei dati e applicazioni mission-critical.  

In kafka abbiamo i seguenti componenti:
1. **Produttori**: inviamp i dati decidendo su quale topic inviarli  
2. **Topic**: Un topic in Kafka è simile a un "canale" o un "argomento" in altri sistemi di messaggistica, Un topic di Kafka è identificato da un nome univoco. I produttori inviano i messaggi al topic specifico e i consumatori leggono i messaggi da quel topic.  
3. **Partizioni***: Un topic può essere suddiviso in più partizioni. Le partizioni consentono di parallelizzare e distribuire l'elaborazione dei messaggi in modo scalabile.  
4. **Consumatori**: Leggono i messaggi da uno o più topic.  

Logstash invia i dati al topic "eurusd" svolgendo il ruolo di produttore mentre Apache Spark svolge il ruolo di consumatore leggendo dal topic.

<img src = "images/topic.png">

# Apache Spark 

Apache Spark è un framework di elaborazione di dati open-source progettato per l'elaborazione distribuita e il calcolo ad alta velocità. È stato sviluppato per affrontare le sfide di scalabilità e prestazioni nell'analisi dei dati su grandi cluster di computer.

Ecco alcune caratteristiche principali di Apache Spark:

1. Velocità: Utilizza la memorizzazione cache per mantenere i dati in memoria, consentendo di accedere rapidamente ai dati senza doverli leggere da un disco. Ciò porta a prestazioni notevolmente superiori rispetto ai sistemi di elaborazione tradizionali basati su disco.

2. Elaborazione distribuita: Spark permette di distribuire il calcolo su un cluster di computer, consentendo di elaborare grandi quantità di dati in parallelo.

3. Ampia compatibilità: Spark supporta diversi linguaggi di programmazione, tra cui Python, Scala, Java e R. Ciò consente agli sviluppatori di utilizzare il linguaggio di programmazione preferito per scrivere applicazioni Spark.


<img src = "images/spark.png">  



Tramite Apache Spark si effettua il collegamento al topic "eurusd" di Kafka per consumare i dati depositati.  

Viene usato Spark MLib per effetture una regressione lineare sui dati relativi al rate e all'orario.

### Regressione Lineare

La regressione lineare cerca di tracciare una linea retta che si adatti meglio ai punti di dati disponibili in modo da poter fare previsioni o stimare il valore della variabile dipendente in base ai valori delle variabili indipendenti.  
L'obiettivo della regressione lineare è trovare i coefficienti che descrivono la migliore linea retta per rappresentare i dati. Questi coefficienti rappresentano l'intercetta (punto in cui la linea interseca l'asse delle ordinate) e il coefficiente angolare (pendenza della linea). Una volta che la linea retta è stata definita, può essere utilizzata per prevedere i valori della variabile dipendente per nuovi dati, utilizzando i valori delle variabili indipendenti. 

```
def linear_regression():
  training_data = spark.createDataFrame([(float(last_refreshed), float(exchange_rate),) for last_refreshed, exchange_rate in zip(timestp, exchange_rates)], ["last_refreshed", "exchange_rate"])
  assembler = VectorAssembler(inputCols=["last_refreshed"], outputCol="features")
  prepared_data = assembler.transform(training_data)

  lr = LinearRegression(featuresCol="features", labelCol="exchange_rate")
  lr_model = lr.fit(prepared_data)

  print("Coefficients: {}".format(lr_model.coefficients))
  print("Intercept: {}".format(lr_model.intercept))

  predictions = lr_model.transform(prepared_data)
  return predictions

```


La funzione linear_regression() è scritta in Python utilizzando la libreria SparkMLib.  
Spark MLlib è una libreria di machine learning di Apache Spark che offre strumenti e algoritmi per l'elaborazione distribuita di dati e l'addestramento di modelli di machine learning su grandi dataset.  


1. Creazione del DataFrame di addestramento:
   - Viene creato un DataFrame utilizzando il metodo spark.createDataFrame().  
   - Il DataFrame è popolato con coppie di valori last_refreshed e exchange_rate. Questi valori rappresentano il timestamp (ultima data di aggiornamento) e il tasso di cambio rispettivamente.  
   - I valori vengono convertiti in formato float utilizzando la funzione float().
   - Viene specificato lo schema dei dati con i nomi delle colonne come "last_refreshed" e    "exchange_rate".

2. Assemblaggio delle feature:
   - Viene creato un oggetto VectorAssembler per assemblare le feature.
   - L'input della fase di assemblaggio sono le colonne "last_refreshed".
   - L'output viene specificato come "features".
   - L'assemblatore viene applicato al DataFrame di addestramento utilizzando il metodo transform() e il risultato viene assegnato a prepared_data.

3. Creazione del modello di regressione lineare:
   - Viene creato un oggetto LinearRegression con featuresCol impostato su "features" e labelCol impostato su "exchange_rate".
        featuresCol indica la colonna contenente le feature utilizzate per l'addestramento del modello.
        labelCol indica la colonna contenente la variabile di output o il target (tasso di cambio).
   - Viene chiamato il metodo fit() sull'oggetto lr per addestrare il modello utilizzando prepared_data.
   - Il modello addestrato viene assegnato a lr_model.

4. Stampare i coefficienti e l'intercetta del modello:
   - Vengono stampati i coefficienti del modello utilizzando lr_model.coefficients.
   - Vengono stampati l'intercetta del modello utilizzando lr_model.intercept.

5. Eseguire previsioni utilizzando il modello:
   - Viene chiamato il metodo transform() sul modello lr_model utilizzando prepared_data.
   - Le previsioni risultanti vengono assegnate a predictions.

6. Restituire le previsioni:
   - La funzione restituisce il DataFrame predictions, che contiene le previsioni del modello per il tasso di cambio in base ai valori di input.


I dati elaborati vengono mandati ad Elasticsearch per l'indicizzazione.


# Elasticsearch  

Elasticsearch è un motore di ricerca e analisi distribuito e open source per tutti i tipi di dati, inclusi testuali, numerici, geospaziali, strutturati e non strutturati. 
Un index in Elasticsearch è una raccolta di documenti correlati tra loro. Elasticsearch archivia i dati come documenti JSON, l'index a cui si inviano i dati è "eurusd".  
I dati vengono passati da Spark dopo aver eseguito la regressione lineare:  

```
doc = {
      "timestamp" : data,
      "timestamp_numeric" : int(last_refreshed_numeric),
      "from_currency_name" : from_currency_name,
      "to_currency_name" : to_currency_name,
      "exchange_rate": exchange_rate,
      "prediction": prediction
    }

    global id
    id = id + 1
      
    risp = es.index(index = es_index, id = id, document = doc)

```

I documenti inviati ad Elasticsearch saranno visualizzabili in: http://localhost:9200/eursd/_doc/{id} 

<img src = "images/meme.jpg">

I dati inviati saranno visualizzati su Kibana.  


# Kibana

Kibana è uno strumento di visualizzazione e gestione dei dati per Elasticsearch che fornisce istogrammi in tempo reale, grafici a linee, grafici a torta e mappe.

# Cosa ho imparato

